# Host Your Own Offline Mapping Server
## Step-by-Step Guide with instructions embedded below
## Overview
<a href="https://leafletjs.com/" target="blank">LeafletJS</a>, <a href="https://www.mapbox.com/" target="blank">MapBox</a>, <a href="https://www.esri.com/" target="blank">Esri</a> etc. These are just a few of the map plugins which analysts, developers etc. who require various Geospatial visualisations tend to use. Needless to say, all these plugins have something in common - they require a basemap service for meaningful interactivity. Thanks to many open-sourced map services such as <a href="https://www.openstreetmap.org/" target="blank">OpenStreetMap(OSM)</a>, it is almost effortless to parse in its map service url for rendering:<br><br>
<u>Simple example of initialising a basemap using LeafletJS</u>
```javascript
    L.tileLayer("https://tile.openstreetmap.org/{z}/{x}/{y}.png", {
        attribution: "Map data &copy; <a href='https://www.openstreetmap.org/'>OpenStreetMap</a> contributors, <a href='https://creativecommons.org/licenses/by-sa/2.0/'>CC-BY-SA</a>, Imagery © <a href='https://www.mapbox.com/'>Mapbox</a>"
    }).addTo(map);
```
## Rationale of hosting an offline basemap
However, this easy accessibility can backfire when an application fails to connect to the map service due to unforeseen circumstances such as - Absence of WiFi, Faulty Internet Connection, the Basemap service is no longer available etc. As a precaution, it is always wiser to consider hosting your own basemap locally on your local server especially on days when you are required to demonstrate or use your application at work.<br><br>
<u>Example of locally hosted basemap</u>
<img src="img/localhost_basemap_preview.png" height="150px" /><br>
Although there are several ways of doing it, my preferred choice is the make use of leaflet's mbtiles plugin known as <a href="https://gitlab.com/IvanSanchez/Leaflet.TileLayer.MBTiles" target="blank">Leaflet.TileLayer.MBTiles</a>, which requires some preparation steps I will be going through below. This includes a little bit of calculation, a local web server, the utility python package <a href="https://github.com/mapbox/mbutil " target="blank">mbutil</a> and the base leafletJS library(since the choice of mbtiles plugin is meant to be integrated with leafletJS).

## Step 1: Search for pre-existing basemap services online and pick your choice
First and foremost, decide on a specific basemap to host offline. A good place to start will be http://maps.stamen.com/ with a great variety of basemaps available for public use. In addition, deciding on which basemap to host enables a clear understanding of each map tile's url format <b>(KIV: This is extremely important as we will see later.)</b>. For demonstration, I shall select MapStamen's Toner basemap. After settling on the type of basemap you wish to host, proceed to:
* Zoom to desired minimum zoom level e.g. `11` and ensure that the current browser viewport is currently capturing **ALL** the map tile images which are rendering the <b>Region/Province/Country/Continent</b>. For example, assuming my objective is to host the basemap of the country, Singapore, my browser shall look like this:<br>
<br><u>Zoom Level 11 View</u><br>
<img src="img/sg_map_toner_hybrid_stamen.png" height="150px" />
* Right click and select "Inspect" or Ctrl+Shift+i (on Windows)
<br><u>Screenshot of Developer Tools</u><br>
<img src="img/browser_console_stamen_basemap.png" height="150px" /><br><br>
While different browsers have different configurations, the 2 essential components are:
1. The browser <b>console</b>
2. The <b>Elements</b> to view the html markup. 
As seen from the format of the `src` attribute of each map tile image rendered,, it follows the conventional <a href="https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames" target="blank">Slippy map</a> format where the URL contains `{s}-{z}/{x}/{y}`:<br>
    - `z`: Zoom Level
    - `x`: X-Coordinate of Tile Image in pixels
    - `y`: Y-Coordinate of Tile Image in pixels
    - `s`: Refers to subdomain. Due to heavy usage of map services, subdomains `a`,`b`,`c` are configured in place.<br>
    
Copy and paste the following JavaScript code snippet into the console:
```javascript
    var imgs = document.getElementsByTagName("img");
    var coordinates = [];
    for(var i in imgs) {
        if(typeof imgs[i].classList !== "undefined") {
            if(imgs[i].classList.contains("leaflet-tile-loaded")) {
                var srcArr=imgs[i].src.split("/");
                var yIndex=srcArr[srcArr.length-1];
                var xIndex=srcArr[srcArr.length-2];
                var zIndex=srcArr[srcArr.length-3];

                var xy={
                    "x": parseInt(xIndex),
                    "y": parseInt(yIndex)
                };

                coordinates.push(xy);
            }
        }
    }
    console.log(JSON.stringify(coordinates));
```
The output shall look similar to this:<br>
<img src="img/javascript_xy_console_output.png" height="150px" /><br>
After running the JavaScript code snippet, all x,y coordinates are retrieved from the image src attribute of each ```<img />``` tag of the basemap at zoom level `11`<br>
<b>Explanation</b>: Since the image src format of each map tile rendered in the basemap follows the following convention: ```http://c.tile.stamen.com/toner-hybrid/11/1614/1016@2x.png``` by delimiting it by the token ```/``` and transforming it into an array, we can see the following result:<br>

| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
|---|---|---|---|---|---|---|
| http: |   | c.tile.stamen.com | toner-hybrid | `11` | `1614` | `1016@2x.png` |

<br><b>Explanation</b>: The top column represents the index of the array. In this case, at zoom level `11`: 
* `x` can be found at position `5`
* `y` can be found at position `6`
Other possible basemap choices from maps.stamen include:<br>

| Type | URL |
|------|-----|
| toner | http://tile.stamen.com/toner/{z}/{x}/{y}.png |
| terrain | http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg |
| watercolor | http://tile.stamen.com/watercolor/{z}/{x}/{y}.jpg |

## Step 2. Import python packages and Initialise functions for Latitude/Longitude or X/Y conversion

In [1]:
import math
from math import pow
import os
import requests

The function `num2deg` transforms the `x` and `y` into `latitude` and `longitude` respectively based on zoom level (`zoom` parameter). Source of code snippet at <a href="https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Lon..2Flat._to_tile_numbers_2" target="blank">Slippy_map_tilenames</a>.

In [2]:
def num2deg(xtile, ytile, zoom):
  n = 2.0 ** zoom
  lon_deg = xtile / n * 360.0 - 180.0
  lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
  lat_deg = math.degrees(lat_rad)
  return (lat_deg, lon_deg)

The function `deg2num` transforms the `lat_deg` and `lon_deg` into `x` and `y` of a tile image respectively based on zoom level (`zoom` parameter) Source of code snippet at <a href="https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Lon..2Flat._to_tile_numbers_2" target="blank">Slippy_map_tilenames</a>.

In [3]:
def deg2num(lat_deg, lon_deg, zoom):
  lat_rad = math.radians(lat_deg)
  n = 2.0 ** zoom
  xtile = int((lon_deg + 180.0) / 360.0 * n)
  ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
  return (xtile, ytile)

### 2.1 Transform all tile coordinates(x,y) into coordinates(latitude,longitude) to combine all markers as a single <a href="https://tools.ietf.org/html/rfc7946#section-3.3" target="blank">GeoJSON Feature Collection</a> object

Copy and Paste the output from Step 1 and store it into the variable `xy`:

In [4]:
xy=[
    {"x":1614,"y":1016},{"x":1615,"y":1016},{"x":1614,"y":1015},
    {"x":1615,"y":1015},{"x":1614,"y":1017},{"x":1615,"y":1017},
    {"x":1613,"y":1016},{"x":1616,"y":1016},{"x":1613,"y":1015},
    {"x":1616,"y":1015},{"x":1613,"y":1017},{"x":1616,"y":1017},
    {"x":1612,"y":1016},{"x":1617,"y":1016},{"x":1612,"y":1015},
    {"x":1617,"y":1015},{"x":1612,"y":1017},{"x":1617,"y":1017}
]

Specify minimum and maximum zoom levels here i.e.The minimum zoom level in Step 1:

In [5]:
minZoomLevel=11 # IMPORTANT! Ensure that your minZoomLevel is the same as the one you decided intially
maxZoomLevel=15 # The upper limit of zoom level set on the basemap

Specify the name of the folder the basemap will be saved in:

In [6]:
directoryPrefix="toner_hybrid/" # e.g. "toner_hybrid"

Initialise a GeoJSON Feature Collection object

In [7]:
geojsonObj={
  "type": "FeatureCollection",
  "features": []
}

<b>KIV</b>: The following 4 variables would be use for later calculations.

In [8]:
minXVal=None
maxXVal=None

minYVal=None
maxYVal=None

In [9]:
for item in xy:
    x=item["x"]
    y=item["y"]
    if (minXVal is None) or (x <= minXVal):
        minXVal=x
        
    if (minYVal is None) or (y <= minYVal):
        minYVal=y
    
    if (maxXVal is None) or (x >= maxXVal):
        maxXVal=x
        
    if (maxYVal is None) or (y >= maxYVal):
        maxYVal=y

    lat=num2deg(x, y, minZoomLevel)[0]
    lng=num2deg(x, y, minZoomLevel)[1]
    feature={
      "type":"Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          lng,
          lat
        ]
       },
        "properties": {
          "zoom": minZoomLevel
        } 
    }
    geojsonObj["features"].append(feature)

Proceed to output the GeoJSON into a file named: `output.geojson`

In [10]:
geojson=str(geojsonObj).replace("'","\"")

geojson_file = open("output.geojson", "w")
geojson_file.write(geojson)
geojson_file.close()

print("GeoJSON has been saved to file: output.geojson")

GeoJSON has been saved to file: output.geojson


In [11]:
# print out min max of x and y
print("Value of minXVal is: " + str(minXVal))
print("Value of maxXVal is: " + str(maxXVal))

print("Value of minYVal is: " + str(minYVal))
print("Value of maxYVal is: " + str(maxYVal))

Value of minXVal is: 1612
Value of maxXVal is: 1617
Value of minYVal is: 1015
Value of maxYVal is: 1017


Retrieve the tile dimensions for minZoomLevel specified i.e. zoom level 11

In [12]:
noOfx={}
noOfy={}

for item in xy:
    x=item["x"]
    y=item["y"]
    noOfx[x]=0
    noOfy[y]=0
    
xTileDimension=len(list(noOfx.keys()))
yTileDimension=len(list(noOfy.keys()))

print("Dimensions of map are: " + str(xTileDimension) + " x " + str(yTileDimension) + " tiles for zoom level " + str(minZoomLevel))

Dimensions of map are: 6 x 3 tiles for zoom level 11


#### 2.2 Visualise map markers (Optional - As proof of tile dimensions)
Render the GeoJSON file onto any map rendering platform to view the coordinates on the actual map. My personal recommendation would be http://geojson.io/ due to its intuitive interface and useful functionalities. Based on the above example, the coordinates transformed from the tiles at zoom level `11` shall look as such:
<img src="img/xy_map_geojson.png" height="150px" /><br>
Take note that the dimensions of the above image shows <b>6 × 3 map markers</b>, which is equivalent to <b>6 × 3 tiles</b>. This corresponds to the calculations above for the tile dimensions.

## Step 3. Calculating the (X,Y) values for other zoom levels

Before proceeding to the actual calculation, it is important to establish the fact that tile coordinates of a slippy map follows the below rules:
* Total no. of tiles=2<sup>zoom</sup> × 2<sup>zoom</sup> E.g. Zoom Level `11` has 2<sup>11</sup> × 2<sup>11</sup> = 2048 × 2048
* Based on the markers visualised on the map, the no. of tiles is: <b>6 × 3 tiles</b> where this corresponds to the dimensions derived earlier on - <b>`xTileDimension` × `yTileDimension` tiles</b>:<br><br>

<b>min(x)</b> = `minXVal` = 1612<br>
<b>max(x)</b> = `maxXVal` =  1617<br>
<br>
<b>min(y)</b> = `minYVal` =  1015<br>
<b>max(y)</b> = `maxYVal` =  1017<br>
<pre>
|----------------
_ x: 1612..1617
|
|
|
|
y:1015..1017
</pre>

Hence, based on the above logic, for zoom level `12`:
* Total no. of tiles=2<sup>12</sup> × 2<sup>12</sup> = 4096 × 4096
* This translates to: (2<sup>12</sup>/2<sup>11</sup>)×`xTileDimension` × (2<sup>12</sup>/2<sup>11</sup>)×`yTileDimension` = (4096/2048)×6 × (4096/2048)×3 tiles = <b>12 × 6 tiles</b> where:<br><br>

<b>min(x)</b> = (`minXVal`/`xTileDimension`) × (xTileDimension of zoom=12) = (1612/6) × 12 = 3224<br>
<b>max(x)</b> = (`maxXVal`/`xTileDimension`) × (xTileDimension of zoom=12) = (1617/6) × 12 = 3234<br>

<br>

<b>min(y)</b> = (`minYVal`/`yTileDimension`) × (yTileDimension of zoom=12) = (1015/3) × 6 = 2030<br>
<b>max(y)</b> = (`maxYVal`/`yTileDimension`) × (yTileDimension of zoom=12) = (1018/3) × 6 = 2036<br>

<pre>
|----------------
_ x: 3224..3234
|
|
|
|
y:2030..2036
</pre>

### 3.1 Proof of Concept that the above equation is valid (optional)

In [13]:
urlPrefix="http://tile.stamen.com/toner-hybrid/"
tileUrls=[]
zoom_level=12
for x in range(3224,3234+1,1):
  for y in range(2030,2036+1,1):
      tileUrl=urlPrefix+str(zoom_level) + "/" + str(x) + "/" + str(y) + ".png"
      tileUrls.append(tileUrl)

Proceed to output some of the tileUrls and select a few links to view the tile image(s).<br>
* If it is a broken link, the map service is probably down so seek another map service.
* Else, the image should render a part of the basemap to be hosted offline subsequently

In [14]:
tileUrls[:12]

['http://tile.stamen.com/toner-hybrid/12/3224/2030.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2031.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2032.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2033.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2034.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2035.png',
 'http://tile.stamen.com/toner-hybrid/12/3224/2036.png',
 'http://tile.stamen.com/toner-hybrid/12/3225/2030.png',
 'http://tile.stamen.com/toner-hybrid/12/3225/2031.png',
 'http://tile.stamen.com/toner-hybrid/12/3225/2032.png',
 'http://tile.stamen.com/toner-hybrid/12/3225/2033.png',
 'http://tile.stamen.com/toner-hybrid/12/3225/2034.png']

From the illustration of the above calculations, it is now possible to calculate and retrieve all the basemap tile images required. Moving on, a folder hierarchy shall be set up in accordance to Slippy map rules to contain the subsequent tile images saved locally.<br><br>
<u>Screenshot of browser backend from the site hosting the map service.</u><br>
<img src="img/basemap_folder_hierarchy.png" width="250px" /><br>
It can be inferred from the above that map tile images are stored in following folder structure:
<pre>
    |--zoom level directory
    |     |---x-tile coordinate directory
    |           |---y-tile coordinate image/png file
</pre>
The next section shall attempt to create the required folder hierarchy. Before executing this, a zoom upper limit of the basemap should be determined. Conventionally, `17` or `18` are good choices when users are required to view street level details. For this tutorial I shall set my zoom level upper limit to just `15`.

### 3.2 Creation of Folder Structure

<b>Recall</b>: At zoom level `12`,<br>
* Total no. of tiles=2<sup>12</sup> × 2<sup>12</sup> = 4096 × 4096
* This translates to: (2<sup>12</sup>/2<sup>11</sup>)×`xTileDimension` × (2<sup>12</sup>/2<sup>11</sup>)×`yTileDimension` = (4096/2048)×6 × (4096/2048)×3 tiles = <b>12 × 6 tiles</b> where:<br><br>

<b>min(x)</b> = (`minXVal`/`xTileDimension`) × (xTileDimension of zoom=12) = (1612/6) × 12 = 3224<br>
<b>max(x)</b> = (`maxXVal`/`xTileDimension`) × (xTileDimension of zoom=12) = (1617/6) × 12 = 3234<br>

<br>

<b>min(y)</b> = (`minYVal`/`yTileDimension`) × (yTileDimension of zoom=12) = (1015/3) × 6 = 2030<br>
<b>max(y)</b> = (`maxYVal`/`yTileDimension`) × (yTileDimension of zoom=12) = (1018/3) × 6 = 2036<br>

<pre>
|----------------
_ x: 3224..3234
|
|
|
|
y:2030..2036
</pre>

<b>There</b>: At zoom level `n`,<br>
* Total no. of tiles=2<sup>n</sup> × 2<sup>n</sup>
* This translates to: <br>
(2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`xTileDimension` × (2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`yTileDimension` where:<br><br>

<b>min(x)</b> = (`minXVal`/`xTileDimension`) × (xTileDimension of zoom=n)<br>
              = (`minXVal`/`xTileDimension`) × (2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`xTileDimension`<br>
              
<b>max(x)</b> = (`maxXVal`/`xTileDimension`) × (xTileDimension of zoom=n)<br>
              = (`maxXVal`/`xTileDimension`) × (2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`xTileDimension`<br>
              
<br>

<b>min(y)</b> = (`minYVal`/`yTileDimension`) × (yTileDimension of zoom=n)<br>
              = (`minYVal`/`yTileDimension`) × (2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`yTileDimension`<br>
              
<b>max(y)</b> = (`maxYVal`/`yTileDimension`) × (yTileDimension of zoom=n)<br>
              = (`maxYVal`/`yTileDimension`) × (2<sup>n</sup>/2<sup>minZoomLevel</sup>)×`yTileDimension`<br>

<pre>
|----------------
_ x: min(x)..max(x)
|
|
|
|
y:min(y)..max(y)
</pre>

To reduce repetitive code, a function shall be created to apply the above logic in order to retrieve that `x` and `y` coordinates of the tiles at any zoom level.

In [20]:
def initDirectoryStructure(n): # where n refers to the zoom level
    minX=(minXVal/xTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*xTileDimension)
    maxX=(maxXVal/xTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*xTileDimension) 
    
    minY=(minYVal/yTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*yTileDimension)
    maxY=(maxYVal/yTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*yTileDimension)
    
    minX=int(minX)
    maxX=int(maxX)
    minY=int(minY)
    maxY=int(maxY)
    
    for x in range(minX,maxX+1,1):
      for y in range(minY,maxY+1,1):
            directory=directoryPrefix+str(n) + "/" + str(x)
            if not os.path.exists(directory):
                os.makedirs(directory)

# Execute function to create folder structures for zoom levels minZoomLevel to maxZoomLevel (inclusive)
for z in range(minZoomLevel,maxZoomLevel+1,1):
    initDirectoryStructure(z)
    
print("Folder structure for zoom levels " + str(minZoomLevel) + " to " + str(maxZoomLevel) + " has been created.")

Folder structure for zoom levels 11 to 15 has been created.


## Step 4. Stream the tile images into the local folders created
After initialisation of the folder structure, we can now retrieve tile images of zoom levels which fall within the range of `minZoomLevel` and `maxZoomLevel` (inclusive) from the basemap service.

### A note of caution: 
1. This step can take up to a few hours if the specified zoom level upper limit is high e.g. Above zoom level `17` will increase the duration of running the next code significantly 
2. The output tile images are going to take up a fair amount of disk space. Generally, raster tiles from map services such as OpenStreetMap comprising of zoom levels `11` to `17` for a single country takes up approximately ~500MB. 
3. If the type of basemap getting hosted is a Satellite map, due to high resolutions of Satellite imagery, the final output often amounts to a few GB so it will be wise to ensure enough disk space is available before saving the tile images to the local disk drive.

In [22]:
basemapUrlPrefix="http://tile.stamen.com/toner-hybrid/"
basemapUrlSuffix=".png"

def streamTileImages(n): # where n refers to the zoom level
    minX=(minXVal/xTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*xTileDimension)
    maxX=(maxXVal/xTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*xTileDimension) 
    
    minY=(minYVal/yTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*yTileDimension)
    maxY=(maxYVal/yTileDimension)*( pow(2,n)/pow(2,minZoomLevel)*yTileDimension)
    
    minX=int(minX)
    maxX=int(maxX)
    minY=int(minY)
    maxY=int(maxY)
    
    for x in range(minX,maxX+1,1):
      for y in range(minY,maxY+1,1):
            # send a HTTP request to retrieve the tile image file
            basemapTileUrl=basemapUrlPrefix+str(n) + "/" + str(x) + "/" + str(y) + basemapUrlSuffix
            r = requests.get(basemapTileUrl)
            # proceed to save it as a local image file in the folders created in step 3
            save_as_filename=directoryPrefix+str(n) + "/" + str(x) + "/" + str(y) + ".png"
            with open(save_as_filename,"wb") as local_tile_image:
                local_tile_image.write(r.content) 

for z in range(minZoomLevel,maxZoomLevel+1,1):
    streamTileImages(z)
    
print("All map tile images for zoom levels " + str(minZoomLevel) + " to " + str(maxZoomLevel) + " have been saved to local directories.")

All map tile images for zoom levels 11 to 15 have been saved to local directories.


### Summary of output from the above steps:
<img src="img/streamed_tile_images_output.png" height="150px" />

### 4.1 Host Folders on a web server to ensure tile images are saved successfully
Finally, to ensure that the correct tile images were saved, a web server should be set up and call the basemap directly from the folders (packaging should only be done after ensuring that the basemap renders correctly).<br>
<br>
For this demonstration, the libraries `Flask` and `werkzeug` shall be used to set up a web server directly in this notebook to host the basemap.<br><br>
<u>Pre-requisites</u>
* Include the base leaflet library: <a href="https://unpkg.com/leaflet@1.0.0/dist/leaflet.css" target="blank">leaflet stylesheet</a> and <a href="https://unpkg.com/leaflet@1.0.0/dist/leaflet-src.js" target="blank">leafletJS</a>
* Include the leaflet plugin <a href="http://ivansanchez.gitlab.io/Leaflet.TileLayer.MBTiles/Leaflet.TileLayer.MBTiles.js" target="blank">L.TileLayer.mbTiles plugin</a> and its dependency <a href="https://unpkg.com/sql.js@0.3.2/js/sql.js" target="blank">sql.js</a>
* Place the file `output.geojson` generated from Step 2 into the `static` folder in the below directory structure specified
* Copy and paste the folder of tile images e.g. <b>toner</b> into the `static/maps` folder in the below directory structure specified

<pre>
    |--Jupyter Notebook (ipynb file)
    |
    |--templates (folder)
    |    |--index.html
    |
    |--static (folder)
    |    |--favicon.ico
    |    |--loading.gif
    |    |--onemap.png
    |    |--output.geojson 
    |    |
    |    |--css (folder)
    |    |   |--leaflet.css
    |    |   |--images (folder)
    |    |        |--layers.png
    |    |        |--layers-2x.png
    |    |        |--marker-icon.png
    |    |   
    |    |--js (folder)
    |    |   |--leaflet.js
    |    |   |--Leaflet.TileLayer.MBTiles.js
    |    |   |--sql.js
    |    |
    |    |--maps (folder) 
    |        |--(folder of tile images e.g.toner)
    |
<pre>

Finally, proceed to include the following code snippet in <b>index.html</b> between the `<script></script>` tags to render the basemap directly from the folders:<br>

```javascript
        var map = new L.Map("map", {
          minZoom: 11,
          maxZoom: 15,
          maxBoundsViscocity: 1
        });
        var basemapLayer=L.tileLayer("static/maps/toner/{z}/{x}/{y}.png", {
          minZoom: 11,
          maxZoom: 15,
          maxBoundsViscocity: 1,
          errorTileUrl: "static/error.png",
          attribution: "<span class='prefix-attribution'><em>Map tiles by <a href='http://stamen.com' target='blank'>Stamen Design</a>, under <a href='http://creativecommons.org/licenses/by/3.0' target='blank'>CC BY 3.0</a>. Data by <a href='http://openstreetmap.org' target='blank'>OpenStreetMap</a>, under <a href='http://www.openstreetmap.org/copyright' target='blank'>ODbL</a>.</span>"
        }).addTo(map);

        var xhr = new XMLHttpRequest();
        xhr.open("GET", "static/output.geojson");
        xhr.setRequestHeader("Content-Type", "application/json");
        xhr.responseType = "json";
        xhr.onload = function() {
            if (xhr.status !== 200) {
              return
            }
            var geoJSONLayer=L.geoJSON(xhr.response);
            map.fitBounds(geoJSONLayer.getBounds());
            map.setMaxBounds(geoJSONLayer.getBounds());
        };
        xhr.send();
```
<b>Explanation of Code Snippet</b>: 
* The above code snippet intialises a new map object with leafletJS. 
* The file <b>output.geojson</b> generated from Step 2 is called via AJAX and proceeds to set the map view within the basemap's bbox.

In [26]:
import requests 

from flask import Flask
from flask import render_template
from flask import request
from werkzeug.wrappers import Request, Response

def shutdown_server():
    func = request.environ.get("werkzeug.server.shutdown")
    if func is None:
        raise RuntimeError("Not running with the Werkzeug Server")
    func()
    
app = Flask(__name__)
@app.route("/")
def index():
    title="Local Basemap"
    return render_template("index.html", message=title, template_folder="templates", static_folder="static")

@app.route("/shutdown", methods=["GET"])
def shutdown():
    shutdown_server()
    return "Server shutting down..."

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost", 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2020 21:01:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:14] "GET /static/js/leaflet-src.map HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2020 21:01:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:24] "GET /static/css/leaflet.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/js/leaflet.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/js/sql.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/js/Leaflet.TileLayer.MBTiles.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/output.geojson HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/maps/toner_hybrid/11/1615/1015.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "GET /static/maps/toner_hybrid/11/1613/1016.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:25] "

127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25831/16256.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25830/16257.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25833/16254.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25832/16256.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25830/16255.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25832/16257.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25832/16255.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25833/16255.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25833/16257.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:01:38] "GET /static/maps/toner_hybrid/15/25

Select the following link to shutdown the Flask Web Application:<br>
<a href="http://localhost:9000/shutdown" target="blank">Shutdown Flask App</a>

Example of initial Basemap View:<br>
<img src="img/initial_basemap_view_toner_hybrid.png" height="150px" /><br>
Zoomed-In View:<br>
<img src="img/zoomed_in_basemap_view_toner_hybrid.png" height="150px" /><br>

### 4.2 Package the folder with `mbutil`
Now that we have ensured the basemap is rendering the correct tile images, the folder can now be packaged into a single <b>mbtiles</b> file. This is done via the python package mbutil. To install it, go to <a href="https://pypi.org/project/mbutil/" target="blank">mbutil</a> and proceed to follow the instructions below:

1. Navigate to <a href="https://github.com/mapbox/mbutil" target="blank">mbutil</a>
2. Download ZIP of the repository and save the ZIP archive into the same folder as the jupyter notebook.
<img src="img/clone_mbutil_folder.png" width="250px" />
3. Extract out the folder `mbutil-master` and rename it to `.mbutil`.
4. Navigate inside the mbutil folder and <b>delete</b> the `.gitignore` file. It should now look like this:
<img src="img/mbutil_package_files.png" width="250px" />
5. Proceed to open a terminal/command prompt in that folder and run: 
<code>python mb-util ../static/maps/toner ../static/maps/toner.mbtiles</code><br><br>
<u>Packaging process by `mbutil` is ongoing</u>
<img src="img/mbutil_packaging_process.png" width="500px" />
6. `toner.mbtiles` is now created and the latest folder structure should now look like this:
<pre>
    |--Jupyter Notebook (ipynb file)
    |
    |--templates (folder)
    |    |--index.html
    |
    |--static (folder)
    |    |--favicon.ico
    |    |--loading.gif
    |    |--loading.gif
    |    |--output.geojson 
    |    |
    |    |--css (folder)
    |    |   |--leaflet.css
    |    |   |--images (folder)
    |    |        |--layers.png
    |    |        |--layers-2x.png
    |    |        |--marker-icon.png
    |    |   
    |    |--js (folder)
    |    |   |--leaflet.js
    |    |   |--Leaflet.TileLayer.MBTiles.js
    |    |   |--sql.js
    |    |
    |    |--maps (folder) 
    |        |--(folder of tile images e.g.toner)
    |        |--(mbtiles file e.g.toner.mbtiles)
    |
    |--.mbutil (folder)
</pre>
Hence, proceed to modify the code snippet in the <b>index.html</b> between the `<script></script>` tags to render the basemap from `toner.mbtiles` file:<br>
```javascript
        var map = new L.Map("map", {
          minZoom: 11,
          maxZoom: 15,
          maxBoundsViscocity: 1
        });

        var basemapLayer = L.tileLayer.mbTiles("static/maps/toner.mbtiles", {
          minZoom: 11,
          maxZoom: 15,
          maxBoundsViscocity: 1,
          attribution: "<span class='prefix-attribution'><em>Map tiles by <a href='http://stamen.com' target='blank'>Stamen Design</a>, under <a href='http://creativecommons.org/licenses/by/3.0' target='blank'>CC BY 3.0</a>. Data by <a href='http://openstreetmap.org' target='blank'>OpenStreetMap</a>, under <a href='http://www.openstreetmap.org/copyright' target='blank'>ODbL</a>.</span>"
        }).addTo(map);

        basemapLayer.on("databaseloaded", function(ev) {
          console.info("MBTiles DB loaded", ev);
                   
          var popup = L.popup();
          
          function onMapClick(e) {
            popup
              .setLatLng(e.latlng)
              .setContent("You clicked the map at " + e.latlng.toString())
              .openOn(map);
          }

          map.on("click", onMapClick);
        });

        basemapLayer.on("databaseerror", function(ev) {
          console.info("MBTiles DB error", ev);
        }); 

        var xhr = new XMLHttpRequest();
        xhr.open("GET", "static/output.geojson");
        xhr.setRequestHeader("Content-Type", "application/json");
        xhr.responseType = "json";
        xhr.onload = function() {
            if (xhr.status !== 200) {
              return
            }
            var geoJSONLayer=L.geoJSON(xhr.response);
            map.fitBounds(geoJSONLayer.getBounds());
            map.setMaxBounds(geoJSONLayer.getBounds());
        };
        xhr.send();
```
<b>Explanation of Code Snippet</b>: 
* The above code snippet is almost identical to the previous code snippet except that instead of rendering the basemap from <b>static/maps/toner/{z}/{x}/{y}.png</b>, the basemap is now rendered from the packaged mbtiles file <b>static/maps/toner.mbtiles</b>

## Step 5. Re-run the web app with the mbtile file and Leaflet.TileLayer.MBTiles.js plugin

In [27]:
def shutdown_server():
    func = request.environ.get("werkzeug.server.shutdown")
    if func is None:
        raise RuntimeError("Not running with the Werkzeug Server")
    func()
    
app = Flask(__name__)
@app.route("/")
def index():
    title="Local Basemap"
    return render_template("index.html", message=title, template_folder="templates", static_folder="static")

@app.route("/shutdown", methods=["GET"])
def shutdown():
    shutdown_server()
    return "Server shutting down..."

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost", 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/css/leaflet.css HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/js/leaflet.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/js/sql.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/js/Leaflet.TileLayer.MBTiles.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/loading.gif HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:15] "GET /static/js/leaflet-src.map HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2020 21:04:16] "GET /static/maps/toner_hybrid.mbtiles HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:16] "GET /static/output.geojson HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:16] "GET /static/favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:16] "GET /static/onemap.png HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 21:04:34] "GET /shutdown H

Select the following link to shutdown the Flask Web Application:<br>
<a href="http://localhost:9000/shutdown" target="blank">Shutdown Flask App</a>

As shown below:
<img src="img/basemap_served_with_mbtiles.png" height="150px" />
The url to all the tile map images are rendered in a binary format fetched from `toner.mbtiles`. Local basemap has now been successfully packaged into a single mbtiles file and deployed onto local server for consumption.<br><br> 

### Congratulations! You now have your basemap rendered on your local server.
#### (Optional) The folder of raw tile images can be deleted (e.g. `static/maps/toner`) and leave only the mbtiles file to be used by the application.